In [34]:
# Autoreload so we're using most recent modules
%load_ext autoreload
%autoreload 2

import sys
import os

# Add the project root to the Python path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(notebook_dir)
sys.path.append(project_root)

# Import from src
from src.data.collection import fetch_games, convert_to_dataframe, fetch_player_stats
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get Games

In [36]:
# Fetch games for 2022 and 2023
games = fetch_games(2022, 2023)
pprint(games[:5])  # Print first 5 games

# Convert to pandas dataframe
df = convert_to_dataframe(games)

# save to parquet
df.to_parquet("../data/games.parquet", index=False)

Successfully fetched games for 2022, conference: SEC
Successfully fetched games for 2022, conference: B1G
Successfully fetched games for 2022, conference: ACC
Successfully fetched games for 2022, conference: B12
Successfully fetched games for 2022, conference: PAC
Successfully fetched games for 2023, conference: SEC
Successfully fetched games for 2023, conference: B1G
Successfully fetched games for 2023, conference: ACC
Successfully fetched games for 2023, conference: B12
Successfully fetched games for 2023, conference: PAC
[{'attendance': 9346,
 'away_conference': 'SEC',
 'away_division': 'fbs',
 'away_id': 238,
 'away_line_scores': [14, 7, 35, 7],
 'away_points': 63,
 'away_post_win_prob': 0.9996410399011048,
 'away_postgame_elo': 1431,
 'away_pregame_elo': 1222,
 'away_team': 'Vanderbilt',
 'completed': True,
 'conference_game': False,
 'excitement_index': 2.4199370723,
 'highlights': None,
 'home_conference': 'Mountain West',
 'home_division': 'fbs',
 'home_id': 62,
 'home_line_sco

## Get Player Game Stats

In [35]:
## Get Player Game Stats

# Fetch player game stats for 2022 and 2023
start_year = 2022
end_year = 2023

for year in range(start_year, end_year + 1):
    player_stats = fetch_player_stats(year)
    
    if player_stats:
        # Convert to dataframe using the existing function
        player_stats_df = convert_to_dataframe(player_stats)
        
        # Display the first few rows
        print(f"Sample data for {year}:")
        print(player_stats_df.head())
        
        # Save to parquet, partitioned by year
        output_path = f"../data/player_game_stats/year={year}"
        player_stats_df.to_parquet(output_path, index=False)
        
        print(f"Successfully fetched and saved player stats for year {year}.")
    else:
        print(f"Failed to fetch player stats for year {year}.")

Successfully fetched player stats for year 2022, week 1
Successfully fetched player stats for year 2022, week 2


KeyboardInterrupt: 

## Get Team Game Stats

## Next

-   Betting Lines
-   Team rankings
-   Drive Data
-   Team Talent Composite
-   Advanced Team Stats
-   Coach Information